In [1]:
import torch
import os
import pandas as pd
from torch_geometric.data import Data

In [2]:
ruta_grafos = r"C:/Users/john/Desktop/Saber_11_2025/data/5_grafos"
ruta_modelos = r"C:/Users/john/Desktop/Saber_11_2025/modelos"
ruta_resultados = r"C:/Users/john/Desktop/Saber_11_2025/resultados"

os.makedirs(ruta_resultados, exist_ok=True)


In [3]:
def cargar_grafos(path, start=2015, end=2022):
    grafos = []
    years = []
    for year in range(start, end+1):
        file = os.path.join(path, f"grafo_{year}_deptos.pt")
        if os.path.exists(file):
            g = torch.load(file, weights_only=False)
            grafos.append(g)
            years.append(year)
    return grafos, years

grafos, years = cargar_grafos(ruta_grafos)
print("Grafos cargados:", years)


Grafos cargados: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


In [6]:
import os

ruta_modelos = r"C:/Users/john/Desktop/Saber_11_2025/notebook_python"

print("Archivos encontrados:")
for f in os.listdir(ruta_modelos):
    if f.endswith(".pt"):
        print(" -", f)


Archivos encontrados:
 - grafo_2015.pt
 - grafo_2016.pt
 - grafo_2017.pt
 - grafo_2018.pt
 - grafo_2019.pt
 - grafo_2020.pt
 - grafo_2021.pt
 - grafo_2022.pt
 - model_2015_sage.pt
 - model_2016_sage.pt
 - model_2017_sage.pt
 - model_2018_sage.pt
 - model_2019_sage.pt
 - model_2020_sage.pt
 - model_2021_sage.pt
 - model_2022_sage.pt


In [4]:
path_model_sage = os.path.join(ruta_modelos, "model_sage.pt")
model_sage = torch.load(path_model_sage, weights_only=False)
model_sage.eval()

print("GraphSAGE cargado")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/john/Desktop/Saber_11_2025/modelos\\model_sage.pt'

In [ ]:
path_model_lstm = os.path.join(ruta_modelos, "model_lstm.pt")
model_lstm = torch.load(path_model_lstm, weights_only=False)
model_lstm.eval()

print("LSTM cargado")


In [ ]:
embeddings = []

for g in grafos:
    with torch.no_grad():
        emb = model_sage(g.x, g.edge_index)  # salida [16]
    embeddings.append(emb)

embeddings = torch.stack(embeddings)  # [8, 16]
print("Embeddings históricos:", embeddings.shape)


In [ ]:
def predecir_futuro(model_sage, model_lstm, grafos, pasos=4):

    # Obtener embeddings reales
    embeds = []
    for g in grafos:
        with torch.no_grad():
            embeds.append(model_sage(g.x, g.edge_index))

    # Usar última ventana (2019–2020–2021–2022 → pero el modelo usa 3)
    ventana = embeds[-3:]  # [2020, 2021, 2022]

    predicciones = []

    for _ in range(pasos):

        x_input = torch.stack(ventana).unsqueeze(0).float()  # [1,3,16]

        with torch.no_grad():
            pred = model_lstm(x_input).squeeze()  # [16]

        predicciones.append(pred)

        # deslizar ventana
        ventana = ventana[1:] + [pred]

    return torch.stack(predicciones)  # [4, 16]


In [ ]:
pred_futuro = predecir_futuro(model_sage, model_lstm, grafos, pasos=4)

print(pred_futuro.shape)
pred_futuro


In [ ]:
path_out = os.path.join(ruta_resultados, "predicciones_2023_2026_embeddings.pt")
torch.save(pred_futuro, path_out)

print("Archivo guardado:", path_out)
